# Blue–Green Deployment with Nginx (Docker)

This repository documents a **Blue/Green deployment architecture** using **Docker** and **Nginx** as a reverse proxy (load balancer).  
The goal is **zero-downtime deployments** with a clean, understandable flow.

---

## 📌 High-Level Architecture

- **1 Nginx container** → acts as a reverse proxy (entry point)
- **2 backend containers**
  - `blue-web` → runs on port **8081**
  - `green-web` → runs on port **8082**
- **1 deploy.sh script** → orchestrates deployments

### Traffic Flow


In [ ]:
Browser
↓
http://localhost
 (Nginx proxy on port 80)
↓
Active backend (Blue or Green)


### Direct Access (for debugging only)
- http://localhost:8081 → Blue container
- http://localhost:8082 → Green container

> ⚠️ In normal usage, you **only access `http://localhost`**.  
> Nginx decides which backend is active.

---

## 🧠 Blue/Green Deployment Logic

The deployment follows this exact sequence:
detect active → deploy inactive → switch proxy → mark new active



### How It Works

1. **Detect active environment**
   - The file `.active_env` contains either:
     ```
     blue
     ```
     or
     ```
     green
     ```

2. **Deploy to inactive environment**
   - If `blue` is active → deploy to `green`
   - If `green` is active → deploy to `blue`

3. **Switch traffic**
   - Nginx upstream configuration is updated to point to the newly deployed container

4. **Mark new environment active**
   - `.active_env` is updated

This ensures:
- No downtime
- Safe rollback (just switch back)

---

## 🧩 Containers Overview

| Container     | Role                  | Port |
|----------------|-----------------------|------|
| nginx-proxy   | Reverse proxy / LB    | 80   |
| blue-web      | Backend (Blue)        | 8081 |
| green-web     | Backend (Green)       | 8082 |

---

## 🔁 Nginx Learning Flow

### Why Nginx?

- Acts as a **single entry point**
- NGINX acts as a reverse proxy by sitting between the clients (e.g., web browsers) and the backend servers, intercepting all incoming requests and routing them to the appropriate server for processing
- Can switch backend traffic instantly
- Perfect for Blue/Green deployments

---

## 📄 Nginx Configuration Structure

Nginx configuration files follow a **strict structure**:

### 1️⃣ `events {}` Block
Configures **how Nginx handles connections internally**.

```nginx
events {
    worker_connections 1024;
}

Each browser tab = one connection

Persistent connections (keep-alive, streaming) stay open

2️⃣ http {} Block

Defines HTTP behavior (reverse proxy, headers, MIME types, etc.)


In [ ]:
http {
    include /etc/nginx/mime.types;
    default_type application/octet-stream;
}

📎 MIME Types Explained
What is mime.types?

A file that maps extensions → content types:

In [ ]:
.html  → text/html
.css   → text/css
.png   → image/png


Why Nginx Needs It

When serving files, Nginx must send:

Content-Type: text/html

If it doesn’t know the type, it falls back to:  application/octet-stream

❌ Result: Browser downloads the file
✅ Correct MIME: Browser displays the content

🧭 Upstream Block (Backend Selection)

The upstream tells Nginx where to send traffic.

In [ ]:
upstream backend {
    server blue-web:80;
    # OR
    server green-web:80;
}

Only one backend is active at a time.

#### 🖥️ Server Block (Virtual Host)

In [ ]:
A server {} block defines one website/service.

server {
    listen 80;
    server_name localhost;

    location / {
        proxy_pass http://backend;
    }
}


Think of it as:

“When traffic arrives on port 80, this is how to handle it.”

## 📍 Location Blocks
#### / → Application Traffic
* - location / {} → Matches all other paths (not exact / but for anything that we put in place of /).

In [ ]:
location / {
    proxy_pass http://backend;
    proxy_http_version 1.1;

    proxy_set_header Host $host;
    proxy_set_header X-Real-IP $remote_addr;
    proxy_set_header X-Forwarded-For $proxy_add_x_forwarded_for;
}


* Forwards requests to the active backend
* Preserves client IP & headers

#### /health → Health Checks

In [ ]:
location /health {
    access_log off;
    add_header Content-Type text/plain;
    return 200 "OK\n";
}


Used by:

Docker

Load balancers

Monitoring systems

📚 Nginx Directives Used
Top-Level Blocks

events {} → connection handling

http {} → HTTP behavior

upstream {} → backend pool

server {} → virtual server

location {} → request routing

Common Directives

worker_connections

include

default_type

listen

server_name

proxy_pass

proxy_set_header

return

add_header

🧠 Mental Model
http://localhost
   ↓
[Nginx Proxy]
   ↓
[ Blue OR Green ]

## <u> Why default.conf(in nginx) File Exists</u>
- Each Blue/Green container is a self‑contained web server.
- default.conf tells Nginx inside those containers:
   - Serve static files from /usr/share/nginx/html.
   - Respond to / with index.html.
   - Handle arbitrary file requests (try_files).
   - Provide /health for monitoring.
- Without this file, the containers wouldn’t know how to serve your HTML or expose a health check.

- **so the nginx.conf file is the default configuration file using which each of the environment will creat their own nginx container**


#### inside this file there is a command called sed 
* Stream EDitor
* A command-line tool used to search, replace, insert, or delete text in files or streams.

**-i**
* In-place editing
* Tells sed to modify the file directly instead of printing the result to the terminal.
* Without -i, the file would stay unchanged.

>"s/server $CURRENT_CONTAINER:80;/server $TARGET_CONTAINER:80;/": This is the sed substitution expression, wrapped in quotes.

**s**
* Means substitute (search and replace).

**/**
* Delimiter that separates parts of the substitution.

* Format is: s / pattern / replacement /

* server $CURRENT_CONTAINER:80; search this exact expression and replace this with server $TARGET_CONTAINER:80;